In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
lev = 600

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat'))
lons = np.asarray(getvar(ctl,'lon'))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure'))

### Height
z_ctl = np.asarray(getvar(ctl,'z'))
z_adj = np.asarray(getvar(adj,'z'))

### Theta
t_ctl = np.asarray(getvar(ctl,'THM')) + TR
t_adj = np.asarray(getvar(adj,'THM')) + TR

### Levels
zl_ctl = interplevel(z_ctl, p_ctl, lev)
zl_adj = interplevel(z_adj, p_ctl, lev)

tl_ctl = interplevel(t_ctl, p_ctl, lev)
tl_adj = interplevel(t_adj, p_ctl, lev)



In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/5

### Plot subdomain
min_lat = 15
max_lat = 62
min_lon = -135
max_lon = -65

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','right']],
                              constrained_layout=True, figsize=(7.4,4.8), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels = False


cs2 = axd['left'].contour(lons, lats, smooth2d(tl_adj,1),
                  levels=np.arange(0,1500,5),linewidths=2.5,
                  colors='r', transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='r')
cs1 = axd['left'].contour(lons, lats, smooth2d(tl_ctl,1),
                  levels=np.arange(0,1500,5),linewidths=1,
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')


# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('Potential Temperature')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

gls = axd['right'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['right'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False
gls.rotate_labels = False


cs4 = axd['right'].contour(lons, lats, smooth2d(zl_adj,2)/10.,
                  levels=np.arange(0,1500,10),linewidths=2.5,
                  colors='r', transform=crs.PlateCarree())
axd['right'].clabel(cs4,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='r')
cs3 = axd['right'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,10),linewidths=1,
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs3,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='k')


# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['right'].set_title('Geopotential Height')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig05_wrf_ics.pdf',bbox_inches='tight')
plt.show()
